In [55]:
import pandas as pd
import numpy as np
import os
import requests
import pickle

import isodate
from datetime import datetime
import time

from dotenv import load_dotenv 
load_dotenv()

# youtube api
from googleapiclient.discovery import build
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors


# text recognition
import nltk
import spacy
import locationtagger
import re
import itertools

# different operators
from collections import Counter
from operator import itemgetter

# pytrends api
#from pytrends.request import TrendReq

# viz
import plotly.express as px


In [5]:
# essential entity models downloads
# nltk.downloader.download('maxent_ne_chunker')
# nltk.downloader.download('words')
# nltk.downloader.download('treebank')
# nltk.downloader.download('maxent_treebank_pos_tagger')
# nltk.downloader.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# !python -m spacy download en_core_web_sm


# Youtube API

https://www.youtube.com/@KaraandNate/videos
Mark Wiens
Yes theory
https://www.youtube.com/woltersworld
https://www.youtube.com/@hopscotchtheglobe/videos
https://www.youtube.com/lostleblanc
https://www.youtube.com/drewbinsky
https://www.youtube.com/@fearlessandfar/videos
https://www.youtube.com/@gonewiththewynns
https://www.youtube.com/@samuelandaudrey

ytp-ce-channel-title ytp-ce-link


In [6]:
channel_ids = ['UCnTsUMBOA8E-OHJE-UrFOnA', #samuel and audrey
               'UC4ijq8Cg-8zQKx8OH12dUSw', #kara and nate
               'UCyEd6QBSgat5kkC6svyjudA', #mark weins
               'UCvK4bOhULCpmLabd2pDMtnA', #yes theory
               'UCFr3sz2t3bDp6Cux08B93KQ', #wolters world
               'UC0Ize0RLIbGdH5x4wI45G-A', #drew binsky
               'UC_DmOS_FBvO4H27U7X0OtRg', #fearless and far
               'UCBo9TLJiZ5HI5CXFsCxOhmA', #gone with the wymns
               'UCt_NLJ4McJlCyYM-dSPRo7Q', #lost leblanc
               'UC0Tf8LUUtL3E24Dr28vXkbA' #hopscotchtheglobe
              ]

In [7]:
# channel_stats = get_channel_stats(youtube, channel_ids)
# channel_stats

In [57]:
channel_stats = pd.read_pickle(r'./data/channel_stats.pkl')
videos_info = pd.read_pickle(r'./data/video_info.pkl')
places_per_year_by_channel = pd.read_pickle(r'./data/places_per_year_by_channel.pkl')
places_per_year_filtered = pd.read_pickle(r'./data/top_places_per_year_views.pkl')

In [9]:
# videos_info = creating_info_df(youtube, channel_stats)

In [10]:
videos_info.dtypes

video_id             object
channelTitle         object
title                object
description          object
tags                 object
publishedAt          object
viewCount           float64
likeCount           float64
commentCount        float64
duration             object
caption              object
duration_minutes      int64
year_published        int64
month_published       int64
day_published         int64
all_countries        object
all_cities           object
all_regions          object
everywhere           object
dtype: object

In [11]:
# videos_details2 = pd.read_pickle('video_details_bestplaces.pkl')
# videos_details = pd.read_pickle('video_details.pkl')

In [12]:
api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.getenv('yt_key')

youtube = googleapiclient.discovery.build(
api_service_name, api_version, developerKey = DEVELOPER_KEY)

In [13]:
def creating_info_df(youtube, channel_data):
    # Create a dataframe with video statistics and comments from all channels

    video_df = pd.DataFrame()
    comments_df = pd.DataFrame()

    for c in channel_data['channelName'].unique():
        print("Getting video information from channel: " + c)
        playlist_id = channel_data.loc[channel_data['channelName']== c, 'playlistId'].iloc[0]
        video_ids = get_video_ids(youtube, playlist_id)

        # get video data
        video_data = get_video_details(youtube, video_ids)
#         # get comment data
#         comments_data = get_comments_in_videos(youtube, video_ids)

        # append video data together and comment data toghether
        video_df = video_df.append(video_data, ignore_index=True)
#         comments_df = comments_df.append(comments_data, ignore_index=True)

    return video_df

In [14]:
def get_channel_stats(youtube, channel_ids):
    
    """
    Get channel stats
    
    Params:
    ------
    youtube: build object of Youtube API
    channel_ids: list of channel IDs
    
    Returns:
    ------
    dataframe with all channel stats for each channel ID
    
    """
    
    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
        
    all_data = pd.DataFrame(all_data)
    
    numeric_cols = ['subscribers', 'views', 'totalVideos']
    all_data[numeric_cols] = all_data[numeric_cols].apply(pd.to_numeric, errors='coerce')
        
    return(all_data)

In [15]:
# # Function to search

# def get_travel_videos_ids(youtube, search):

#     request = youtube.search().list(
    
#     part="id,snippet",
#     maxResults=50,
#     order="relevance",
#     q=search,
#     relevanceLanguage="en",
#     type="video",
#     videoCategoryId="19"
#     )

#     response = request.execute()

#     all_videos = []

#     for i in range(len(response['items'])):

#         all_videos.append(response['items'][i]['id']['videoId'])

#     next_page_token = response.get('nextPageToken')
#     more_pages = True

#     while more_pages:
#         if next_page_token is None:
#             more_pages = False
#         else:
#             request = youtube.search().list(

#             part="id,snippet",
#             maxResults=50,
#             order="relevance",
#             q=search,
#             relevanceLanguage="en",
#             type="video",
#             videoCategoryId="19",
#             pageToken = next_page_token
#             )

#             response = request.execute()

#             for i in range(len(response['items'])):
#                 all_videos.append(response['items'][i]['id']['videoId'])

#             next_page_token = response.get('nextPageToken')
                
#     return all_videos


In [16]:
def get_video_ids(youtube, playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel
    
    Returns:
    List of video IDs of all videos in the playlist
    
    """
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

In [17]:
def get_video_details(youtube, video_ids):
    
    """
    Get video statistics of all videos with given IDs
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'caption'(boolean)
    """
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {
                             'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'commentCount'],
                             'contentDetails': ['duration', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)


In [18]:
#all_videos = get_travel_videos_ids(youtube, "travel|trip|vacation")
#len(all_videos)

In [19]:
#all_videos2 = get_travel_videos_ids(youtube, "best place to visit")
# len(all_videos2)

In [20]:
# videos_details = get_video_details(youtube, all_videos)
# videos_details2 = get_video_details(youtube, all_videos2)

In [21]:
# videos_details2.sample(3)

In [22]:
# videos_details2.dtypes

In [23]:
# videos_details2.isna().sum()

## Cleaning and using entity recognition

In [24]:
def cleaning_dfs(video_df):
    
    # convert count columns to numeric
    numeric_cols = ['viewCount', 'likeCount', 'commentCount']
    video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

    # convert duration to minutes
    video_df['duration_minutes'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x).seconds // 60)
    
    # convert date from ISO format and isolate year, month and day
    pd.to_datetime(video_df['publishedAt'])
    video_df['year_published'] = pd.DatetimeIndex(video_df['publishedAt']).year
    video_df['month_published'] = pd.DatetimeIndex(video_df['publishedAt']).month
    video_df['day_published'] = pd.DatetimeIndex(video_df['publishedAt']).day


In [25]:
def deleting_emojis(video_df):
    
    video_df['title'] = video_df['title'].str.replace('[^\w\s#@/:%.,_-]', '', regex=True, flags=re.UNICODE).str.replace('#', '',)
    video_df['title'] = video_df['title'].map(lambda x: x.lower() if isinstance(x,str) else x)
    
    video_df['description'] = video_df['description'].str.replace('[^\w\s#@/:%.,_-]', '', regex=True, flags=re.UNICODE).str.replace('#', '',)
    video_df['description'] = video_df['description'].map(lambda x: x.lower() if isinstance(x,str) else x)
    
    video_df['tags'] = video_df['tags'].astype(str).str.replace('[^\w\s#@/:%.,_-]', '', regex=True, flags=re.UNICODE).str.replace('#', '',)
    video_df['tags'] = video_df['tags'].map(lambda x: x.lower() if isinstance(x,str) else x)
    

In [26]:
# deleting_emojis(videos_info)

In [27]:
# cleaning_dfs(videos_info)

In [28]:
videos_info.sample(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,caption,duration_minutes,year_published,month_published,day_published,all_countries,all_cities,all_regions,everywhere
370,EN_VQoa1tb8,Drew Binsky,the forgotten jews of ethiopia,subscribe for daily travel videos: http://bit...,"ethiopia, ethiopian jews, gondar, drew binsky ...",2020-01-04T15:00:02Z,1248461.0,29115.0,5747.0,PT8M53S,false,8,2020,1,4,"[afghanistan, philippines, israel, ethiopia, t...",[],[Georgia],"[Afghanistan, Philippines, Israel, Ethiopia, T..."
6306,er_Kyj0pyAw,Wolters World,millennial travel advice for traveling with th...,how to travel with your adult parents. how to ...,"travel, europe, wolters world, travel advice, ...",2022-11-23T13:00:33Z,4257.0,314.0,45.0,PT8M34S,false,8,2022,11,23,[finland],"[Finland, Turku]",[],"[Finland, Turku]"
3160,Sax7lFD89cc,Samuel and Audrey - Travel and Food Videos,"eating a deep-fried mars bar in edinburgh, sco...","on a quest to try a deep-fried mars bar, we fo...","deep-fried mars bar dish, deep fried mars bar,...",2014-01-01T07:30:00Z,317631.0,1750.0,528.0,PT4M6S,true,4,2014,1,1,[],"[Edinburgh, Scotland, Four]",[Edinburgh],"[Edinburgh, Scotland, Four]"
6997,2pTRN_ZLzZw,Wolters World,getting a degree abroad: 10 things to know bef...,getting your degree abroad can be an incredibl...,"travel, europe, mark wolters, wolters world, t...",2018-08-20T12:00:00Z,17835.0,982.0,164.0,PT9M2S,false,9,2018,8,20,[portugal],"[Four, Us, Lisbon]",[Lisbon],"[Portugal, Four, Us, Lisbon]"
6413,GYH6_7OM7LU,Wolters World,"bratislava: the donts of visiting bratislava, ...",hey there fellow travelers welcome to bratisla...,"travel, europe, wolters world, travel advice, ...",2022-03-16T12:00:06Z,87276.0,2576.0,395.0,PT9M4S,false,9,2022,3,16,[slovakia],"[Bratislava, Vienna, Budapest]",[Vienna],"[Slovakia, Bratislava, Vienna, Budapest]"


In [29]:
def extracting_places(video_df):
    
    places_title = []
    places_tags = []
    places_description = []
    all_countries = []
    all_cities = []
    all_regions = []
    
    for index, row in video_df.iterrows():
        
        # extracting entities
        if row['title']:
            place_entity_title = locationtagger.find_locations(text = str(row['title']))
            
        else:
            place_entity_title = locationtagger.find_locations(text = 'none')


        if row['tags']:
            place_entity_tags = locationtagger.find_locations(text = str(row['tags']))
            
        else:
            place_entity_tags = locationtagger.find_locations(text = 'none')
          
        if row['description']:
            place_entity_description = locationtagger.find_locations(text = str(row['description']))
        
        else:
            place_entity_description = locationtagger.find_locations(text = 'none')

#         getting all countries, states and/or cities
#         places_title.append({'countries': place_entity_title.countries,
#                         'states': place_entity_title.regions,
#                         'cities': place_entity_title.cities})
        
#         places_tags.append({'countries': place_entity_tags.countries,
#                         'states': place_entity_tags.regions,
#                         'cities': place_entity_tags.cities})
        
#         places_description.append({'countries': place_entity_description.countries,
#                         'states': place_entity_description.regions,
#                         'cities': place_entity_description.cities})
            
        all_countries.append([place_entity_title.countries, place_entity_tags.countries, place_entity_description.countries])
        all_cities.append([place_entity_title.cities, place_entity_tags.cities, place_entity_description.cities])
        all_regions.append([place_entity_title.regions, place_entity_tags.regions, place_entity_description.regions])
        
        print(index)
        
#     video_df['places_title'] = places_title
#     video_df['places_tags'] = places_tags
#     video_df['places_description'] = places_description
    
    for i in range(len(all_countries)):
        
        all_countries[i] = list(set(itertools.chain(*all_countries[i])))  
        all_cities[i] = list(set(itertools.chain(*all_cities[i])))
        all_regions[i] = list(set(itertools.chain(*all_regions[i])))
        
    video_df['all_countries'] = all_countries
    video_df['all_cities'] = all_cities
    video_df['all_regions'] = all_regions
    
    video_df['everywhere'] = (video_df['all_countries'] + video_df['all_cities'] + video_df['all_regions'])
    video_df['everywhere'] = video_df['everywhere'].apply(lambda x: list(pd.Series(x).str.capitalize().unique()) if x else None)



In [30]:
videos_info.shape

(9512, 19)

In [31]:
#extracting_places(videos_info)

In [32]:
videos_info_copy = videos_info.copy()

In [33]:
# videos_details2.to_pickle('video_details_bestplaces.pkl')
# videos_details.to_pickle('video_details.pkl')

In [58]:

def deleting_biased_places(videos_info, to_delete, query, channel_title, year_):
    
    if query == 1:
        
        for idx, row in videos_info.iterrows():
            
            if ((row['channelTitle'] == channel_title) & (row['everywhere'] != None)):
                
                for i in to_delete:
                    
                    if (i in row['everywhere']):

                        row['everywhere'].remove(i)
    
    else:
        for idx, row in videos_info.iterrows():
            
            if ((row['channelTitle'] == channel_title) & (row['year_published'] >= year_) & (row['everywhere'] != None)):
                
                for i in to_delete:
                    
                    if (i in row['everywhere']):

                        row['everywhere'].remove(i)
        

In [35]:
#deleting_biased_places(videos_info, ['Tennessee', 'Nashville'], 1, 'Kara and Nate', 0)
#deleting_biased_places(videos_info, ['Ontario', 'Grimsby'], 2, 'Kristen & Siya', 2017)
# deleting_biased_places(videos_info, ['Four', 'Ed', 'Thomas'], 1, 'Yes Theory', 0)
#deleting_biased_places(videos_info, ['Pakistan'], 2, 'Drew Binsky', 2021)

In [36]:
# places_per_year_by_channel = counting_ocurrences_places_by_channel(videos_info)

In [37]:
# kara and nate: tennessee and nashville are in every description 
# kristen e siya: their address is on many descriptions and it's grimsby, ontario(17, 18, 19); they do have videos talking about ontario though
# yes theory: ed, four and thomas are not places
# drew binsky: in some of his recent (21, 22) descriptions he links some of his most popular videos and one of them is "► Why is Everything Free in Pakistan?"


places_per_year_by_channel.loc[places_per_year_by_channel['channel'] == 'Drew Binsky']

,channel,year,countries,cities,regions,everywhere
102,Drew Binsky,2012,[],"[(Prague, 1)]","[(Wisconsin, 1)]","[(Wisconsin, 1), (Prague, 1)]"
103,Drew Binsky,2013,"[(israel, 1), (nicaragua, 1), (japan, 1)]","[(Seoul, 2), (English, 1), (Tel Aviv, 1), (Gra...","[(Seoul, 2), (Tel Aviv, 1), (Granada, 1)]","[(Seoul, 2), (English, 1), (Israel, 1), (Tel a..."
104,Drew Binsky,2014,"[(australia, 1), (indonesia, 1), (hong kong, 1)]","[(Seoul, 10), (English, 3), (Village, 1), (Mac...","[(Seoul, 10), (Taiwan, 1), (Taipei, 1)]","[(Seoul, 10), (English, 3), (Village, 1), (Mac..."
105,Drew Binsky,2015,"[(india, 1), (myanmar, 1)]","[(English, 5), (Seoul, 2), (Yangon, 2), (Mumba...","[(Seoul, 2)]","[(English, 5), (Seoul, 2), (Yangon, 2), (India..."
106,Drew Binsky,2016,"[(hong kong, 1), (germany, 1), (latvia, 1)]","[(Hanoi, 1), (Scottsdale, 1), (Hong Kong, 1), ...","[(Arizona, 1), (Taipei, 1)]","[(Hanoi, 1), (Scottsdale, 1), (Arizona, 1), (H..."
107,Drew Binsky,2017,"[(philippines, 2), (hong kong, 2), (japan, 1),...","[(Manila, 2), (Hong Kong, 2), (Seoul, 2), (Eng...","[(Seoul, 2), (Pyongyang, 2), (Taipei, 1), (Ari...","[(Philippines, 2), (Hong kong, 2), (Manila, 2)..."
108,Drew Binsky,2018,"[(thailand, 51), (philippines, 27), (india, 13...","[(Bangkok, 37), (English, 19), (Michael, 17), ...","[(Bangkok, 37), (Arizona, 9), (New York, 7), (...","[(Thailand, 51), (Bangkok, 37), (Philippines, ..."
109,Drew Binsky,2019,"[(india, 33), (pakistan, 30), (thailand, 21), ...","[(Michael, 24), (Manila, 20), (Syria, 19), (Ba...","[(Bangkok, 15), (Kabul, 10), (Madrid, 8), (Ker...","[(India, 33), (Pakistan, 30), (Michael, 24), (..."
110,Drew Binsky,2020,"[(afghanistan, 21), (mexico, 18), (philippines...","[(Mexico, 18), (Kabul, 15), (Turkey, 13), (Ame...","[(Kabul, 15), (Arizona, 9), (Mexico City, 9), ...","[(Afghanistan, 21), (Mexico, 18), (Philippines..."
111,Drew Binsky,2021,"[(pakistan, 44), (iraq, 14), (afghanistan, 10)...","[(Baghdad, 9), (Caracas, 8), (Jamaica, 7), (Sy...","[(Dubai, 4), (Kingston, 2), (Pyongyang, 2), (A...","[(Iraq, 14), (Afghanistan, 10), (Baghdad, 9), ..."


In [59]:
def counting_ocurrences_places(video_df):

    # I didn't get only the top 3 or 4 places automatically because the entitity recognition library is not very
    # precise and I needed to select by hand which places are real places and which are a possible mistake from the 
    # locationtagger library
    
    everywhere = []
    all_countries = []
    all_cities = []
    all_regions = []
    places_per_year = pd.DataFrame()

    cities = []
    everywhere_ = []
    countries = []
    regions = []
    year = []

    for i in range(video_df['year_published'].min(), video_df['year_published'].max()+1):

        vd = video_df.loc[video_df['year_published']==i]

        for j in range(len(vd["everywhere"])):

            if vd["everywhere"].iloc[j]:
                everywhere += vd["everywhere"].iloc[j] 

            if vd["all_countries"].iloc[j]:
                all_countries += vd["all_countries"].iloc[j]

            if vd["all_cities"].iloc[j]:
                all_cities += vd["all_cities"].iloc[j]

            if vd["all_regions"].iloc[j]:
                all_regions += vd["all_regions"].iloc[j]
        
        year.append(i)
        countries.append(Counter(all_countries).most_common())
        cities.append(Counter(all_cities).most_common())
        regions.append(Counter(all_regions).most_common())
        everywhere_.append(Counter(everywhere).most_common())
        
        everywhere = []
        all_countries = []
        all_cities = []
        all_regions = []
        
    places_per_year['year'] = year
    places_per_year['countries'] = countries
    places_per_year['cities'] = cities
    places_per_year['regions'] = regions
    places_per_year['everywhere'] = everywhere_

        
    return places_per_year

In [60]:
# refactor using the "everything_string" column and groupby with .count()

def counting_ocurrences_places_by_channel(video_df):

    # I didn't get only the top 3 or 4 places automatically because the entitity recognition library is not very
    # precise and I needed to select by hand which places are real places and which are a possible mistake from the 
    # locationtagger library
    
    everywhere = []
    all_countries = []
    all_cities = []
    all_regions = []
    places_per_year = pd.DataFrame()

    cities = []
    everywhere_ = []
    countries = []
    regions = []
    year = []
    channel = []

    for c in video_df['channelTitle'].unique():
        df = video_df.loc[video_df['channelTitle']==c]
        
        for i in range(df['year_published'].min(), df['year_published'].max()+1):

            vd = df.loc[df['year_published']==i]

            for j in range(len(vd["everywhere"])):

                if vd["everywhere"].iloc[j]:
                    everywhere += vd["everywhere"].iloc[j] 

                if vd["all_countries"].iloc[j]:
                    all_countries += vd["all_countries"].iloc[j]

                if vd["all_cities"].iloc[j]:
                    all_cities += vd["all_cities"].iloc[j]

                if vd["all_regions"].iloc[j]:
                    all_regions += vd["all_regions"].iloc[j]

            
            channel.append(c)
            year.append(i)
            countries.append(Counter(all_countries).most_common())
            cities.append(Counter(all_cities).most_common())
            regions.append(Counter(all_regions).most_common())
            everywhere_.append(Counter(everywhere).most_common())
            
            everywhere = []
            all_countries = []
            all_cities = []
            all_regions = []
        
    places_per_year['channel'] = channel
    places_per_year['year'] = year
    places_per_year['countries'] = countries
    places_per_year['cities'] = cities
    places_per_year['regions'] = regions
    places_per_year['everywhere'] = everywhere_

        
    return places_per_year

In [40]:
places_per_year_by_channel.loc[places_per_year_by_channel['channel'] == 'Samuel and Audrey - Travel and Food Videos']

,channel,year,countries,cities,regions,everywhere
21,Samuel and Audrey - Travel and Food Videos,2012,"[(canada, 8), (china, 4), (malaysia, 2), (indi...","[(Seoul, 14), (Canada, 8), (English, 5), (Chin...","[(Seoul, 14), (Alberta, 1), (New York, 1)]","[(Seoul, 14), (Canada, 8), (English, 5), (Chin..."
22,Samuel and Audrey - Travel and Food Videos,2013,"[(malaysia, 49), (cambodia, 40), (thailand, 35...","[(English, 33), (China, 29), (Chiang Mai, 24),...","[(Kuala Lumpur, 17), (Bangkok, 14), (Seoul, 14...","[(Malaysia, 49), (Cambodia, 40), (Thailand, 35..."
23,Samuel and Audrey - Travel and Food Videos,2014,"[(canada, 21), (thailand, 19), (germany, 13), ...","[(Canada, 21), (Canadian, 17), (Finland, 12), ...","[(Queensland, 9), (Ontario, 8), (Bangkok, 7), ...","[(Canada, 21), (Thailand, 19), (Canadian, 17),..."
24,Samuel and Audrey - Travel and Food Videos,2015,"[(canada, 29), (peru, 26), (south africa, 18),...","[(Canada, 29), (Peru, 26), (Lima, 24), (Canadi...","[(Lima, 24), (Paris, 11), (Milan, 8), (Madrid,...","[(Canada, 29), (Peru, 26), (Lima, 24), (Canadi..."
25,Samuel and Audrey - Travel and Food Videos,2016,"[(argentina, 44), (peru, 41), (brazil, 16), (c...","[(Seoul, 44), (Argentina, 44), (Peru, 41), (Li...","[(Seoul, 44), (Lima, 27), (Edinburgh, 7), (Lis...","[(Seoul, 44), (Argentina, 44), (Peru, 41), (Li..."
26,Samuel and Audrey - Travel and Food Videos,2017,"[(india, 19), (hong kong, 16), (poland, 15), (...","[(Four, 19), (Hong Kong, 16), (Poland, 15), (C...","[(Taiwan, 13), (Vienna, 12), (Taipei, 12), (Ku...","[(Four, 19), (India, 19), (Hong kong, 16), (Po..."
27,Samuel and Audrey - Travel and Food Videos,2018,"[(japan, 37), (italy, 35), (canada, 22), (peru...","[(Italy, 35), (Tokyo, 25), (Canada, 22), (Peru...","[(Lima, 18), (Venice, 8), (Rome, 8), (Naples, ...","[(Japan, 37), (Italy, 35), (Tokyo, 25), (Canad..."
28,Samuel and Audrey - Travel and Food Videos,2019,"[(canada, 92), (japan, 34), (argentina, 23), (...","[(Canada, 92), (Seoul, 34), (Tokyo, 30), (Arge...","[(Seoul, 34), (Cordoba, 21), (Nara, 15), (Brit...","[(Canada, 92), (Seoul, 34), (Japan, 34), (Toky..."
29,Samuel and Audrey - Travel and Food Videos,2020,"[(canada, 69), (argentina, 49), (india, 17), (...","[(Canada, 69), (Argentina, 49), (Atlantic, 38)...","[(Cordoba, 36), (Ontario, 5), (New Brunswick, ...","[(Canada, 69), (Argentina, 49), (Atlantic, 38)..."
30,Samuel and Audrey - Travel and Food Videos,2021,"[(argentina, 42), (germany, 11), (canada, 10),...","[(Argentina, 42), (German, 11), (Canada, 10), ...","[(Ontario, 6), (Cordoba, 4), (Bavaria, 3), (Co...","[(Argentina, 42), (Germany, 11), (German, 11),..."


In [82]:
# so I can use the organizing function to get rid of the tuple and to be able to load this df into sql
places_per_year_by_channel.rename(columns={'everywhere':'top_places'}, inplace=True)

In [ ]:
def organizing_places_per_channel_df(places_per_year_by_channel):

    places1 = []
    mentions = []
    channel = []
    year = []
    aux = pd.DataFrame()

    for idx, row in places_per_year_by_channel.iterrows():
        
        for c in row['channel'].unique():
            
            for i in row['top_places']:

                channel.append(c)
                year.append(row['year'])
                places1.append(i[0])
                mentions.append(i[1])

        for j in row['views_per_place']:
            if row['year'] == year:

                places2.append(j[0])
                n_views.append(j[1])
                
    aux['top_places_'] = places1
    aux['mentions'] = mentions
    aux['total_number_of_views'] = n_views
    aux['year'] = year
    aux
    
    return aux.sort_values(by = ['total_number_of_views'], ascending=False)

In [75]:
#places_per_year = counting_ocurrences_places(videos_info)
places_per_year.to_pickle(r'./data/all_places_views_per_year')

In [66]:
places_per_year.rename(columns={'everywhere':'top_places'}, inplace=True)

In [67]:
places_per_year.sample(2)

,year,countries,cities,regions,top_places
8,2017,"[(canada, 132), (thailand, 75), (mexico, 44), ...","[(Tennessee, 159), (Nashville, 156), (Four, 14...","[(Tennessee, 159), (Ontario, 60), (Bangkok, 40...","[(Canada, 132), (Los angeles, 81), (Montreal, ..."
4,2013,"[(thailand, 71), (china, 64), (india, 60), (ma...","[(Bangkok, 66), (China, 64), (English, 49), (C...","[(Bangkok, 66), (Kuala Lumpur, 18), (Seoul, 16...","[(Thailand, 71), (Bangkok, 66), (China, 64), (..."


In [64]:
places_per_year['everywhere'].loc[12]

[('Four', 50),
 ('Argentina', 44),
 ('Thailand', 41),
 ('Us', 23),
 ('California', 23),
 ('New zealand', 22),
 ('Canada', 21),
 ('Iraq', 21),
 ('Los angeles', 18),
 ('Afghanistan', 18),
 ('Indonesia', 16),
 ('Christmas', 15),
 ('Germany', 15),
 ('United states', 14),
 ('Las vegas', 14),
 ('Santa monica', 14),
 ('America', 13),
 ('Italy', 12),
 ('Turkey', 12),
 ('German', 12),
 ('Tonga', 12),
 ('China', 11),
 ('Arizona', 10),
 ('Baghdad', 10),
 ('New york', 9),
 ('Louisiana', 9),
 ('Nevada', 9),
 ('France', 9),
 ('Iowa', 8),
 ('Bangkok', 8),
 ('Mauritania', 8),
 ('English', 8),
 ('Rex', 8),
 ('Ecuador', 8),
 ('Caracas', 8),
 ('San francisco', 7),
 ('New orleans', 7),
 ('Atlantic', 7),
 ('Tucson', 7),
 ('El calafate', 7),
 ('Philippines', 7),
 ('Costa rica', 7),
 ('Kenya', 7),
 ('Saudi arabia', 7),
 ('Jamaica', 7),
 ('Texas', 6),
 ('Jeffrey', 6),
 ('Mexico', 6),
 ('Philadelphia', 6),
 ('India', 6),
 ('Chile', 6),
 ('Spanish', 6),
 ('Ontario', 6),
 ('Pakistan', 6),
 ('Florida', 6),
 ('Ore

In [43]:
# in 2020 there were too many videos about argentina from one single channel that isn't very big, so it's important to check qty of views for each place

# places_per_year_filtered = pd.DataFrame()
# places_per_year_filtered['year'] = [2018, 2019, 2020, 2021]
# places_per_year_filtered['top_places'] = [list(itemgetter(1,2,3,4)(places_per_year['everywhere'].iloc[9])),
#                                            list(itemgetter(1,2,3,4)(places_per_year['everywhere'].iloc[10])),
#                                            list(itemgetter(1,2,3,7,8)(places_per_year['everywhere'].iloc[11])),
#                                            list(itemgetter(1,2,3,4,5)(places_per_year['everywhere'].iloc[12]))]


In [44]:
places_per_year_filtered

,year,top_places,views_per_place
0,2018,"[(Thailand, 92), (Philippines, 71), (Bangkok, ...","[(Thailand, 125362547.0), (Philippines, 464475..."
1,2019,"[(Canada, 100), (Japan, 61), (Thailand, 52), (...","[(Canada, 5785997.0), (Japan, 35180294.0), (Th..."
2,2020,"[(Canada, 96), (Argentina, 56), (Thailand, 51)...","[(Canada, 9177054.0), (Argentina, 2734348.0), ..."
3,2021,"[(Argentina, 44), (Thailand, 41), (Us, 23), (C...","[(Argentina, 1428384.0), (Thailand, 78097246.0..."


In [72]:
def get_views_per_top_place(video_df, places_df):
    
    views = 0
    total_views_per_place = []
    total_views_per_place2 = []
  
    for idx, row in places_df.iterrows():
        for place in row['top_places']:

            for idx2, row2 in video_df.iterrows():

                if row2['year_published'] == row['year']:

                    if (row2['everywhere'] != None):
                        if (place[0] in row2['everywhere']):
                            views += row2['viewCount']

            total_views_per_place.append((place[0], views))
            views = 0


        total_views_per_place2.append(total_views_per_place)
        total_views_per_place = []

    places_df['views_per_place'] = total_views_per_place2
        
    return places_df

In [43]:
def organizing_places_views_df(places_per_year_filtered, year):

    places1 = []
    mentions = []
    places2 = []
    n_views = []
    aux = pd.DataFrame()

    for idx, row in places_per_year_filtered.iterrows():
        for i in row['top_places']:
            if row['year'] == year:

                places1.append(i[0])
                mentions.append(i[1])

        for j in row['views_per_place']:
            if row['year'] == year:

                places2.append(j[0])
                n_views.append(j[1])
                
    aux['top_places_'] = places1
    aux['mentions'] = mentions
    aux['total_number_of_views'] = n_views
    aux['year'] = year
    
    return aux.sort_values(by = ['total_number_of_views'], ascending=False)

In [51]:
aux = organizing_places_views_df(places_per_year_filtered, 2021)
places_per_year_filtered2 = pd.concat([places_per_year_filtered2, aux]).reset_index(drop = True)
places_per_year_filtered2

,top_places_,mentions,total_number_of_views,year
0,Thailand,92,125362547.0,2018
1,Bangkok,64,90442073.0,2018
2,Philippines,71,46447584.0,2018
3,Japan,57,17499578.0,2018
4,Thailand,52,72046161.0,2019
5,India,52,60987692.0,2019
6,Japan,61,35180294.0,2019
7,Thailand,51,100217920.0,2020
8,India,35,42043903.0,2020
9,Canada,96,9177054.0,2020


In [46]:
# places_per_year_filtered = get_views_per_top_place(videos_info, places_per_year_filtered)
places_per_year_filtered['views_per_place'].loc[3]

[('Argentina', 1428384.0),
 ('Thailand', 78097246.0),
 ('Us', 17452715.0),
 ('California', 26217356.0),
 ('New zealand', 7581476.0)]

In [73]:
places_per_year = get_views_per_top_place(videos_info, places_per_year)
places_per_year.sample()

,year,countries,cities,regions,top_places,views_per_place
3,2012,"[(thailand, 31), (italy, 25), (france, 15), (m...","[(Bangkok, 35), (Italy, 25), (Seoul, 22), (Kua...","[(Bangkok, 35), (Seoul, 22), (Kuala Lumpur, 10...","[(Bangkok, 35), (Thailand, 31), (Italy, 25), (...","[(Bangkok, 14561163.0), (Thailand, 12239246.0)..."


In [91]:
places_per_year2.to_pickle(r'./data/all_places_views_per_year')

In [90]:
aux2 = organizing_places_views_df(places_per_year, 2010)
places_per_year2 = pd.concat([places_per_year2, aux2]).reset_index(drop = True)
places_per_year2

,top_places_,mentions,total_number_of_views,year
0,Thailand,92,125362547.0,2018
1,Pakistan,17,103972401.0,2018
2,Bangkok,64,90442073.0,2018
3,Mexico,40,82375702.0,2018
4,Ghana,10,48081699.0,2018
...,...,...,...,...
4166,Pisa,1,1265.0,2010
4167,Quel,1,1265.0,2010
4168,Charles,1,840.0,2010
4169,Grammer,1,576.0,2010


In [323]:
# leaving only the top 3 places by total number of views
## I later realized "Us" is probably a problem with the locationtagger; every time it read "us" it recognized as The US
# so I had to change this df in the "finished" notebook and substitute Us for New Zealand
places_per_year_filtered['views_per_place'][0].remove(('Japan', 17499578.0))
places_per_year_filtered['views_per_place'][0]

[('Thailand', 125362547.0),
 ('Philippines', 46447584.0),
 ('Bangkok', 90442073.0)]

In [48]:
# print(places_per_year_by_channel['everywhere'].loc[places_per_year_by_channel['year']==2018])

In [49]:
# vd = videos_info.loc[videos_info['year_published']==2018]
# vd['everywhere']=vd['everywhere'].astype('str')
# vd = vd.groupby(['channelTitle']).agg(places = ('everywhere', 'unique'))


In [50]:
# for idx, row in places_per_year_by_channel.iterrows():
#     print(row['channel'], pd.Series(row['everywhere']).str.contains('German', regex=False).sum())

In [51]:
# channel_stats.to_pickle('channel_stats.pkl')
# videos_info.to_pickle('videos_info.pkl')
# videos_info.to_pickle('video_info.pkl')

In [162]:
# places_per_year_by_channel.to_pickle('places_per_year_by_channel.pkl')
# places_per_year_filtered.to_pickle('top_places_per_year_views.pkl')

# Pytrends API

In [300]:
def getting_google_trends(places_df, year):
    
    pytrends = TrendReq(hl='en-US', tz=360)
    kw_list = []
    
    for idx, row in places_df.iterrows():
        if row['year'] == year:
            
            for i in row['views_per_place']:

                kw_list.append(i[0])
               
            pytrends.build_payload(kw_list, timeframe=f'{year}-1-1 {year}-12-31')
                
    return pytrends.interest_over_time()

In [183]:
pytrends_top_places_2021 = getting_google_trends(places_per_year_filtered, 2021)
pytrends_top_places_2021

,Thailand,The U.S,California,isPartial
date,,,,
2021-01-03,16,0,68,False
2021-01-10,19,0,69,False
2021-01-17,21,0,70,False
2021-01-24,17,0,74,False
2021-01-31,16,0,67,False
2021-02-07,15,0,65,False
2021-02-14,16,0,69,False
2021-02-21,15,0,69,False
2021-02-28,16,0,69,False


In [186]:
pytrends_top_places_2019 = getting_google_trends(places_per_year_filtered, 2019)
pytrends_top_places_2019

,Japan,Thailand,India,isPartial
date,,,,
2019-01-06,13,7,62,False
2019-01-13,12,6,65,False
2019-01-20,12,5,61,False
2019-01-27,13,6,60,False
2019-02-03,12,5,63,False
2019-02-10,12,5,57,False
2019-02-17,12,5,58,False
2019-02-24,12,6,81,False
2019-03-03,12,5,68,False


In [299]:
# theus2021 = pd.read_csv('theus2021.csv', skiprows = 2)
# pd.DataFrame(theus2021).reset_index(drop=True, inplace=True)
# pytrends_top_places_2021['The U.S'] = theus2021['The U.S: (Todo o mundo)'].to_list()
# pytrends_top_places_2021

,Thailand,The U.S,California,isPartial
date,,,,
2021-01-03,16,68,68,False
2021-01-10,19,85,69,False
2021-01-17,21,87,70,False
2021-01-24,17,92,74,False
2021-01-31,16,90,67,False
2021-02-07,15,91,65,False
2021-02-14,16,81,69,False
2021-02-21,15,93,69,False
2021-02-28,16,98,69,False


In [281]:
#pytrends_top_places_2020 = getting_google_trends(places_per_year_filtered, 2020)
#pytrends_top_places_2020['Thailand'] = thailand2020['Thailand']
# canada2020 = pd.read_csv('canada2020.csv', skiprows = 2)

pytrends_top_places_2020['Canada'] = canada2020.to_list()
pytrends_top_places_2020


# pd.DataFrame(canada2020).reset_index(drop=True, inplace=True)
# canada2020.to_list()

,Canada,Thailand,India,isPartial
date,,,,
2020-01-05,77,91,71,False
2020-01-12,73,94,78,False
2020-01-19,72,91,73,False
2020-01-26,70,94,87,False
2020-02-02,71,100,100,False
2020-02-09,66,93,69,False
2020-02-16,68,84,65,False
2020-02-23,74,91,71,False
2020-03-01,72,93,66,False


In [295]:
# japan2019 = pd.read_csv('japan2019.csv', skiprows = 2)
# pd.DataFrame(japan2019).reset_index(drop=True, inplace=True)
# pytrends_top_places_2019['Japan'] = japan2019['Japan: (Todo o mundo)'].to_list()
# pytrends_top_places_2019

,Japan,Thailand,India,isPartial
date,,,,
2019-01-06,70,7,62,False
2019-01-13,69,6,65,False
2019-01-20,66,5,61,False
2019-01-27,70,6,60,False
2019-02-03,65,5,63,False
2019-02-10,68,5,57,False
2019-02-17,69,5,58,False
2019-02-24,65,6,81,False
2019-03-03,68,5,68,False


In [298]:
# thailand2019 = pd.read_csv('thailand2019.csv', skiprows = 2)
# pd.DataFrame(thailand2019).reset_index(drop=True, inplace=True)
# pytrends_top_places_2019['Thailand'] = thailand2019['Thailand: (Todo o mundo)'].to_list()
# pytrends_top_places_2019

,Japan,Thailand,India,isPartial
date,,,,
2019-01-06,70,100,62,False
2019-01-13,69,88,65,False
2019-01-20,66,82,61,False
2019-01-27,70,86,60,False
2019-02-03,65,83,63,False
2019-02-10,68,79,57,False
2019-02-17,69,80,58,False
2019-02-24,65,98,81,False
2019-03-03,68,81,68,False


In [330]:
# pytrends_top_places_2018 = pd.DataFrame()
# thailand2018 = pd.read_pickle('thailand_pytrends_2018.pkl')
# thailand2018
# pytrends_top_places_2018['date'] = thailand2018['date']
# pytrends_top_places_2018['Thailand'] = thailand2018['Thailand']

# pytrends_top_places_2018

,date,Thailand
0,2018-01-07,30
1,2018-01-14,29
2,2018-01-21,27
3,2018-01-28,26
4,2018-02-04,26
5,2018-02-11,26
6,2018-02-18,25
7,2018-02-25,27
8,2018-03-04,25
9,2018-03-11,25


In [331]:
# bangkok2018 = pd.read_csv('bangkok2018.csv', skiprows = 2)
# pd.DataFrame(bangkok2018).reset_index(drop=True, inplace=True)
# pytrends_top_places_2018['Bangkok'] = bangkok2018['Bangkok: (Todo o mundo)'].to_list()

# pytrends_top_places_2018

,date,Thailand,Bangkok
0,2018-01-07,30,99
1,2018-01-14,29,99
2,2018-01-21,27,90
3,2018-01-28,26,95
4,2018-02-04,26,85
5,2018-02-11,26,85
6,2018-02-18,25,87
7,2018-02-25,27,84
8,2018-03-04,25,86
9,2018-03-11,25,84


In [332]:
# philippines2018 = pd.read_csv('philippines2018.csv', skiprows = 2)
# pd.DataFrame(philippines2018).reset_index(drop=True, inplace=True)
# pytrends_top_places_2018['Philippines'] = philippines2018['Philippines: (Todo o mundo)'].to_list()

# pytrends_top_places_2018

,date,Thailand,Bangkok,Philippines
0,2018-01-07,30,99,89
1,2018-01-14,29,99,91
2,2018-01-21,27,90,92
3,2018-01-28,26,95,88
4,2018-02-04,26,85,83
5,2018-02-11,26,85,82
6,2018-02-18,25,87,84
7,2018-02-25,27,84,86
8,2018-03-04,25,86,84
9,2018-03-11,25,84,83


In [333]:
# pytrends_top_places_2020.to_pickle('pytrends_top_places_2020.pkl')
# pytrends_top_places_2018.to_pickle('pytrends_top_places_2018.pkl')
# pytrends_top_places_2019.to_pickle('pytrends_top_places_2019.pkl')
# pytrends_top_places_2021.to_pickle('pytrends_top_places_2021.pkl')

pytrends_top_places_2020 = read_pickle('pytrends_top_places_2020.pkl')
pytrends_top_places_2018=read_pickle('pytrends_top_places_2018.pkl')
pytrends_top_places_2019=read_pickle('pytrends_top_places_2019.pkl')
pytrends_top_places_2021=read_pickle('pytrends_top_places_2021.pkl')

In [145]:
#data = pytrends.interest_over_time() 
#data2.reset_index(inplace=True, drop=True)
#data2.drop(columns=['level_0', 'index'], inplace = True)
#data2.to_pickle('thailand_pytrends_2018.pkl')

In [100]:
# data2 = data
# data.reset_index(inplace=True)
# data.dtypes
# data['month'] = pd.DatetimeIndex(data['date'].month)
# data

AttributeError: 'Series' object has no attribute 'month'

In [143]:

# fig = px.line(data2, x='date', y='Thailand', title='Keyword Web Search Interest Over Time')
# fig.show() 

In [144]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

select = videos_info_copy.loc[(videos_info_copy['everywhere'].str.contains("Thailand")) & (videos_info_copy['year_published'] == 2018)]

trace1 = go.Line(
    x=data2['date'],
    y=data2['Thailand'],
    name='Interest over time for Thailand',
    marker=dict(
        color='rgb(34,163,192)'
               )
)
trace2 = go.Scatter(
    x=select['publishedAt'],
    y=select['viewCount'],
    name='Videos posted by number of views',
    yaxis='y2',
    mode = 'markers'

)

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1)
fig.add_trace(trace2, secondary_y=True)
fig['layout'].update(height = 600, width = 800, 
                     title = "Google Trends for 'Thailand' x Videos posted on YT that mention 'Thailand'",
                     xaxis=dict(
      tickangle=-90
    ))

fig.show()

C:\Users\mayar\miniconda3\envs\final_project\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [80]:
videos_info_copy['everywhere'] = videos_info_copy['everywhere'].astype('str').str.replace('[', '', regex=True).str.replace(']', '', regex=True)

In [334]:
videos_info['everywhere_string'] = videos_info['everywhere'].astype('str').str.replace('[', '', regex=True).str.replace(']', '', regex=True)



In [37]:
videos_info.sample()

NameError: name 'videos_info' is not defined

In [133]:
       
select = videos_info_copy.loc[(videos_info_copy['everywhere'].str.contains("Thailand")) & (videos_info_copy['year_published'] == 2018)]
select

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,caption,duration_minutes,year_published,month_published,day_published,all_countries,all_cities,all_regions,everywhere
276,SKuJkHDRJdM,Kara and Nate,bugs incredible thai street food chiang mai ...,check out the travel app we created that hel...,"chiang mai sunday night market, chiang mai, su...",2018-12-21,244004.0,5831.0,572.0,PT17M12S,false,17,2018,Dec,21,[thailand],"[Chiang Mai, Tennessee, Nashville]",[Tennessee],"'Thailand', 'Chiang mai'"
277,o2l59pRPT8U,Kara and Nate,"top 13 must eat foods in chiang mai, thailand ...",we created an app to help you save up to 80%...,"chiang mai food, chiang mai, thai street food,...",2018-12-19,402622.0,8080.0,539.0,PT13M41S,false,13,2018,Dec,19,[thailand],"[Chiang Mai, Tennessee, Nashville]",[Tennessee],"'Thailand', 'Chiang mai'"
278,esb-rdjGKeM,Kara and Nate,living in chiang mai 750 apartment tour thai...,want hidden flight deals to international de...,"chiang mai, digital nomad, thailand visa, thai...",2018-12-17,357480.0,7309.0,380.0,PT12M41S,false,12,2018,Dec,17,[thailand],"[Chiang Mai, Tennessee, Nashville]",[Tennessee],"'Thailand', 'Chiang mai'"
279,6INw4PJ7ybQ,Kara and Nate,how to stay in hotels for free singapore to ch...,we created faredrop to help you score super ...,"thai lion air, lion air, singapore, singapore ...",2018-12-15,298180.0,5978.0,394.0,PT12M10S,false,12,2018,Dec,15,"[thailand, singapore, canada]","[Canada, Chiang Mai, Singapore, Tennessee, Nas...",[Tennessee],"'Thailand', 'Singapore', 'Canada', 'Chiang mai'"
357,Ya070qqAj0Q,Kara and Nate,thailand to myanmar on air aisa nates birthda...,check out the travel app we created that hel...,"shwedagon pagoda, travel vlog, myanmar country...",2018-04-26,363495.0,7881.0,1053.0,PT13M26S,false,13,2018,Apr,26,"[myanmar, thailand]","[Bangkok, Yangon, Tennessee, Nashville]","[Bangkok, Tennessee]","'Myanmar', 'Thailand', 'Bangkok', 'Yangon'"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,CwbjULHMFso,Drew Binsky,why we moved to bangkok,my just go app is here connect w/ travelers f...,"bangkok, thailand, bangkok city, bangkok thail...",2018-07-17,375398.0,7545.0,715.0,PT2M57S,false,2,2018,Jul,17,"[thailand, philippines]",[Bangkok],[Bangkok],"'Thailand', 'Philippines', 'Bangkok'"
794,zMFRvtkTSHo,Drew Binsky,"today, the world won 13 boys saved in thai cave",my just go app is here connect w/ travelers f...,"thailand cave rescue, thailand, thai cave resc...",2018-07-10,49421.0,2044.0,174.0,PT1M58S,false,1,2018,Jul,10,[thailand],[Bangkok],[Bangkok],"'Thailand', 'Bangkok'"
829,zHddatcNDDE,Drew Binsky,travel with us to thailand group trip: nov 18,my just go app is here connect w/ travelers f...,"thailand, group trip, budget travel, thailand ...",2018-06-04,50417.0,1092.0,149.0,PT2M32S,false,2,2018,Jun,4,[thailand],"[Chiang Mai, English]",[],"'Thailand', 'Chiang mai', 'English'"
842,v5sXqU0sga0,Drew Binsky,the worlds most addictive drug betel nut,my just go app is here connect w/ travelers f...,"betel nut, drug, addictive, travel, asia, sout...",2018-05-15,7355357.0,99057.0,14143.0,PT4M39S,false,4,2018,May,15,[thailand],[],[],'Thailand'


In [131]:
videos_info_copy.dtypes
#videos_info_copy['month_published'] = pd.to_datetime(videos_info_copy['month_published'], format='%m').dt.month_name().str[:3]
videos_info_copy['publishedAt'] = pd.to_datetime(videos_info_copy['publishedAt']).dt.strftime('%Y-%m-%d')

In [132]:
videos_info_copy.sample()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,caption,duration_minutes,year_published,month_published,day_published,all_countries,all_cities,all_regions,everywhere
6781,_N1-hadtw78,Wolters World,"lima: the best worst of visiting lima, peru",grab some of our gear http://www.woltersworld....,"travel, europe, mark wolters, wolters world, t...",2020-01-18,41096.0,1144.0,126.0,PT7M43S,false,7,2020,Jan,18,[peru],"[Four, Peru, Lima]",[Lima],"'Peru', 'Four', 'Lima'"


In [6]:
places_per_year_filtered = pd.read_pickle(r'./data/top_places_per_year_views.pkl')


In [15]:
places2

['Thailand', 'Philippines', 'Bangkok', 'Japan']

In [50]:
places_per_year_filtered['top_places'][3].remove(('Argentina', 44))

ValueError: list.remove(x): x not in list

In [44]:
places_per_year_filtered2 = pd.DataFrame()

In [22]:

places_per_year_filtered2['top_places_'] = places1
places_per_year_filtered2['mentions'] = mentions
places_per_year_filtered2['total_number_of_views'] = n_views
places_per_year_filtered2['year'] = 2018
places_per_year_filtered2

,top_places_,mentions,total_number_of_views,year
0,Thailand,92,125362547.0,2018
1,Philippines,71,46447584.0,2018
2,Bangkok,64,90442073.0,2018
3,Japan,57,17499578.0,2018


In [54]:
places_per_year_filtered2.to_pickle(r"./data/places_year_filtered_notuple.pkl")

# Tourism data

In [38]:
#number_of_tourist_arrivals.to_pickle('international-tourism-number-of-arrivals.pkl')
number_of_tourist_arrivals = pd.read_pickle('international-tourism-number-of-arrivals.pkl')


In [22]:
# Complementing missing data with results from the internet: thailand (2020 and 2021 - statista),
# japan (2021 - https://statistics.jnto.go.jp), philippines (2021 - statista), canada (2020 and 2021 - www150.statcan.gc.ca)
# usa (it was only missing 2021, but upon comparing this data to the one provided by the gov, https://www.trade.gov/, 
# the values were really different, so I decided to update them by using the government's ones - only from 2018 on)
# bangkok (2017-2019), new zealand(2021 - https://www.stats.govt.nz/information-releases/international-travel-december-2021/)
# california (2018-2021, https://industry.visitcalifornia.com/research/travel-forecast)
# arizona (2018-20213)

new_rows = pd.DataFrame({'Entity' : ['Thailand', 'Thailand', 'Philippines', 'Canada', 'Canada', 'Japan', 'United States', 
                        'Bangkok', 'Bangkok', 'Bangkok', 'New zealand'],
           'Code' : ['THA', 'THA', 'PHL', 'CAN', 'CAN', 'JPN', 'USA', 'THA-BKK', 'THA-BKK', 'THA-BKK','NZL'],
            'Year': [2020, 2021, 2021, 2021, 2020, 2021, 2021, 
                     2018, 2017, 2019, 2021],
            'International tourism, number of arrivals': [6700000, 430000, 160000, 3062214, 2959885, 245862, 22100000, 
                                                          23000000, 20800000, 25850000, 206862]
           })

number_of_tourist_arrivals = pd.concat([new_rows, number_of_tourist_arrivals]).reset_index(drop=True)

In [ ]:
new_rows = pd.DataFrame({'Entity' : ['California', 'California', 'California', 'California', 'Arizona', 'Arizona',
                        'Arizona', 'Arizona'],
           'Code' : ['USA-CA', 'USA-CA', 'USA-CA', 'USA-CA', 'USA-AZ', 'USA-AZ', 'USA-AZ', 'USA-AZ'],
            'Year': [2018, 2019, 2020, 2021, 2018, 2019, 2020, 2021],
            'International tourism, number of arrivals': [18100000, 17900000, 3700000, 5200000, 45400000, 46800000, 32100000, 
                                                          40900000]
           })

number_of_tourist_arrivals = pd.concat([new_rows, number_of_tourist_arrivals]).reset_index(drop=True)

In [34]:
number_of_tourist_arrivals.at[4791,'International tourism, number of arrivals'] = 80400000

In [35]:
number_of_tourist_arrivals[number_of_tourist_arrivals['Code'] == 'USA']

,Entity,Code,Year,"International tourism, number of arrivals"
6,United States,USA,2021,22100000.0
4768,United States,USA,1995,79732000.0
4769,United States,USA,1996,82756000.0
4770,United States,USA,1997,82525000.0
4771,United States,USA,1998,74767000.0
4772,United States,USA,1999,75796000.0
4773,United States,USA,2000,78343000.0
4774,United States,USA,2001,70975000.0
4775,United States,USA,2002,64434000.0
4776,United States,USA,2003,62082000.0


In [4]:
# the csv file was cleaned and edited by me to include only the places analysed here;
# missing data was also included when possible (philippines - manila (2020) https://www.philstar.com/business/2021/02/10/2076517/low-hotel-occupancy-persist-year)
# bangkok (2017 - 2021)

hotel_data = pd.read_excel('hotel_data_edited.xlsx')
hotel_data.to_pickle('hotel_data_edited.pkl')

In [5]:
hotel_data.sample(5)

,year,country,number_of_establishments,number_of_rooms,number_of_bed_places,occupancy_rate_rooms,occupancy_rate _bed_places,average_length_of_stay
26,2021,thailand,..,..,..,14.16,..,..
123,2005,india,1190,67613,135226,67.7,..,..
150,2005,canada,..,..,..,..,59.9,..
143,1998,canada,..,330003,..,..,64,5.22
13,2008,thailand,..,361193,..,50.23,..,..


In [ ]:
## to degub - sql on streamlit
import mysql.connector
# Initialize connection.
# Uses st.experimental_singleton to only run once.
@st.experimental_singleton
def init_connection():
    return mysql.connector.connect(**st.secrets["mysql"])

conn = init_connection()

# Perform query.
# Uses st.experimental_memo to only rerun when the query changes or after 10 min.
@st.experimental_memo(ttl=600)
def run_query(query):
    with conn.cursor() as cur:
        cur.execute(query)
        return cur.fetchall()

videos_info = run_query("SELECT * FROM info_about_all_videos;")